<a href="https://colab.research.google.com/github/Vyoma-garg/Natural-Language-Processing/blob/main/Torch_Text_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 7.0MB 7.7MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
from torchtext.data import Field, TabularDataset, BucketIterator

#from torchtext.legacy import data 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [16]:
######### Loading from JSON/CSV/TSV files #########

# STEPS:
# 1. Specify how preprocessing should be done -> Fields
# 2. Use Dataset to load the data -> TabularDataset (JSON/CSV/TSV Files)
# 3. Construct an iterator to do batching & padding -> BucketIterator


!pip install spacy 
!python -m spacy download en
spacy_en = spacy.load("en")


def tokenize(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

#tokenize=lambda x : x.split()

quote = Field(sequential=True, use_vocab=True, tokenize=tokenize, lower=True)
score = Field(sequential=False, use_vocab=False)

fields = {"quote": ("q", quote), "score": ("s", score)}

train_data, test_data = TabularDataset.splits(
    path="/content/drive/MyDrive/Custom_Text_dataset", train="train.json", test="test.json", format="json", fields=fields
)

# # train_data, test_data = TabularDataset.splits(
# #                                         path='mydata',
# #                                         train='train.csv',
# #                                         test='test.csv',
# #                                         format='csv',
# #                                         fields=fields)

# # train_data, test_data = TabularDataset.splits(
# #                                         path='mydata',
# #                                         train='train.tsv',
# #                                         test='test.tsv',
# #                                         format='tsv',
# #                                         fields=fields)

print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())
print(train_data[1].__dict__.values())
print(train_data[2].__dict__.values())

quote.build_vocab(train_data, max_size=10000, min_freq=1, 
                  vectors="glove.6B.100d"
                  )

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), batch_size=2, device=device
)

for batch in train_iterator:
  print(batch.q)  #torch text pads the sequences itself where there is no token as 1
  print(batch.s)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:13: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
.vector_cache/glove.6B.zip:   0%|          | 8.19k/862M [00:00<5:00:21, 47.8kB/s]

dict_keys(['q', 's'])
dict_values([['you', 'must', 'own', 'everything', 'in', 'your', 'world', '.', 'there', 'is', 'no', 'one', 'else', 'to', 'blame', '.'], 1])
dict_values([['do', 'not', 'pray', 'for', 'an', 'easy', 'life', ',', 'pray', 'for', 'the', 'strength', 'to', 'endure', 'a', 'difficult', 'one', '.'], 1])
dict_values([['stand', 'tall', ',', 'and', 'rice', 'like', 'a', 'potato', '!'], 0])


.vector_cache/glove.6B.zip: 862MB [02:39, 5.39MB/s]                           
 99%|█████████▉| 397748/400000 [00:17<00:00, 27263.30it/s]

tensor([[35],
        [23],
        [26],
        [18],
        [19],
        [36],
        [34],
        [ 2],
        [33],
        [20],
        [24],
        [ 6],
        [16],
        [ 8],
        [12],
        [ 2]])
tensor([1])
tensor([[29, 14],
        [31, 25],
        [ 3,  7],
        [11,  5],
        [28, 10],
        [22, 15],
        [ 4, 21],
        [27,  3],
        [ 9,  7],
        [ 1,  5],
        [ 1, 32],
        [ 1, 30],
        [ 1,  8],
        [ 1, 17],
        [ 1,  4],
        [ 1, 13],
        [ 1,  6],
        [ 1,  2]])
tensor([0, 1])


/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [17]:
######### Training a simple LSTM on this toy data of ours #########
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers)
        self.fc_out = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device)

        embedded = self.embedding(x)
        outputs, _ = self.rnn(embedded, (h0, c0))
        prediction = self.fc_out(outputs[-1, :, :])

        return prediction

In [19]:
 len(quote.vocab)

37

In [18]:
# Hyperparameters
input_size = len(quote.vocab)
hidden_size = 512
num_layers = 2
embedding_size = 100
learning_rate = 0.005
num_epochs = 10

In [20]:
# Initialize network
model = RNN_LSTM(input_size, embedding_size, hidden_size, num_layers).to(device)

In [24]:
# (NOT COVERED IN YOUTUBE VIDEO): Load the pretrained embeddings onto our model
pretrained_embeddings = quote.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
print(pretrained_embeddings.shape)
print(pretrained_embeddings)

torch.Size([37, 100])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.4918,  1.1164,  1.1424,  ..., -0.5088,  0.6256,  0.4392],
        [-0.4989,  0.7660,  0.8975,  ..., -0.4118,  0.4054,  0.7850],
        [-0.5718,  0.0463,  0.8673,  ..., -0.3566,  0.9293,  0.8995]])


In [25]:
# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
# Train Network
for epoch in range(num_epochs):
    losses=[]
    for batch_idx, batch in enumerate(train_iterator):
        # Get data to cuda if possible
        data = batch.q.to(device=device)
        targets = batch.s.to(device=device)

        # forward
        scores = model(data)
        #print(scores.shape)
        loss = criterion(scores.squeeze(1), targets.type_as(scores))
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()
    print(f"Loss at epoch {epoch} is { sum(losses)/len(losses) :.5f}") 

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Loss at epoch 0 is 0.55137
Loss at epoch 1 is 0.48148
Loss at epoch 2 is 0.36660
Loss at epoch 3 is 0.23677
Loss at epoch 4 is 0.11191
Loss at epoch 5 is 0.04681
Loss at epoch 6 is 2.41923
Loss at epoch 7 is 1.75828
Loss at epoch 8 is 0.83441
Loss at epoch 9 is 0.20301


In [36]:
def check_accuracy(loader,model):
  if loader.dataset.train:
        print('Checking Accuracy on Training Data')
  else:
        print('Checking Accuracy on Test Data')
  num_correct=0
  num_samples=0
  model.eval()

  for batch_idx, batch in enumerate(train_iterator):
      x = batch.q.to(device=device)
      y = batch.s.to(device=device)
    
      scores=model(x)  
      values,predictions =scores.max(1)  #interested in index of the max value
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0) 

  print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

  model.train()

check_accuracy(train_iterator, model)
check_accuracy(test_iterator, model)



Checking Accuracy on Training Data
Got 1/ 3 with accuracy 33.33 

Checking Accuracy on Training Data
Got 1/ 3 with accuracy 33.33 



/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
